<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       In-Database Complaints Analysis Integration with Customer360 using LLMs
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>
<p style='font-size:16px;font-family:Arial'>Complaints Analysis Integration with Customer360 is a comprehensive approach to managing customer complaints and feedback within the framework of a Customer 360-degree view using <b>Teradata Vantage</b> and <b>Amazon Bedrock</b>. This integration aims to provide a seamless and personalized customer experience by leveraging data from various sources, including CRM systems, marketing platforms, and social media.</p> <p style='font-size:16px;font-family:Arial'>The key components of this integration include:</p> 

<ol style='font-size:16px;font-family:Arial'> <li><b>Customer 360 Data Manager</b>: Responsible for managing and maintaining a comprehensive view of customer data, including collection, integration, and analysis from multiple sources.</li> <li><b>Complaints Management Dashboard</b>: Analyzes customer complaints, providing insights into complaint volume, trends, and resolution progress.</li> <li><b>Customer Insights</b>: Tools for gaining insights into customer behavior and preferences, enabling targeted marketing campaigns and informed business decisions.</li> </ol> <p style='font-size:16px;font-family:Arial'>The benefits of this integration include:</p> <ol style='font-size:16px;font-family:Arial'> <li><b>Improved Customer Experience</b>: By integrating complaints analysis with Customer 360, businesses can address customer complaints more effectively, leading to increased customer satisfaction and loyalty.</li> <li><b>Data-Driven Decision Making</b>: The integration provides a centralized platform for analyzing customer data, enabling businesses to make informed decisions about product development, marketing strategies, and customer engagement.</li> <li><b>Enhanced Customer Insights</b>: The comprehensive view of customer data allows businesses to better understand customer needs and preferences, leading to more targeted and effective marketing efforts.</li> </ol> 


<p style='font-size:16px;font-family:Arial'>By integrating complaints analysis with Customer 360, businesses can create a more comprehensive and personalized customer experience, driving business growth and customer satisfaction.</p> 

<p style = 'font-size:16px;font-family:Arial'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Connect to Vantage</li>
    <li>Configure server-side LLM access using Teradatagenai package</li>
    <li>Execute in-database Sentiment Analysis, Topic Modelling and Complaint Summarization</li>
    <li>Integrate data with customer 360</li>
    <li>Cleanup</li>
</ol>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial;'><b>Download and install additional software as needed</b>

In [ ]:
%pip install --upgrade -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style='height:1px;border:none'>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import numpy as np
import pandas as pd
import json, warnings
import getpass
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi


# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.max_rows = 5
pd.set_option('display.max_colwidth', None)
display.suppress_vantage_runtime_warnings = True

<hr style='height:2px;border:none'>
<b style = 'font-size:20px;font-family:Arial'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>Connection information has been defined in an external file - adjust as necessary.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=text_analytics_teradatagenai_aws_huggingface.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>2.  Set up the LLM connection</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use anthropoc claude-instant-v1 for low-cost, high-throughput tasks.</p>

<ol style = 'font-size:16px;font-family:Arial'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
access_key = getpass.getpass('aws_access_key_id: ')
secret_key = getpass.getpass('aws_secret_access_key: ')
region_name = getpass.getpass('region name: ')

<hr style='height:2px;border:none'>
<p style='font-size:20px;font-family:Arial'><b>3. Use the <code>TextAnalyticsAI</code> API to Perform Various Text Analytics Tasks</b></p>
<p style='font-size:16px;font-family:Arial'>You can execute the help function at the bottom of this notebook to read more about this API.</p>

In [ ]:
# Provide model details
model_name="us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = region_name,
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)

obj = TextAnalyticsAI(llm=llm)

<hr style='height:2px;border:none'>
<b style = 'font-size:20px;font-family:Arial'>4. Using LLM for Sentiment Analysis, Topic Modeling and Complaint Summarization</b>

<p style = 'font-size:16px;font-family:Arial'>Sentiment Analysis, Topic Modeling and Complaint Summarization using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style='font-size:16px;font-family:Arial'>Sentiment Analysis Using Large Language Models (LLMs) is a cutting-edge approach to understanding customer opinions and emotions expressed through text-based data. This advanced technique leverages the capabilities of LLMs to accurately identify and categorize sentiment as positive, negative, or neutral, providing businesses with valuable insights into customer perceptions and preferences.</p>

<p style = 'font-size:16px;font-family:Arial'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<hr style='height:1px;border:none'>
<p style = 'font-size:18px;font-family:Arial'><b>4.1 Inspect source data</b></p>

<p style = 'font-size:16px;font-family:Arial'>The Teradata python package (teradataml) allows users to work with data using common python syntax and methods <b>without</b> moving data to the client - all operations are pushed to the MPP platform, allowing rapid, performant analytics on data at any scale.  In this case, the DataFrame object represents a table or query <b>in-database</b> which could contain millions or billions of records.</p>

In [ ]:
customer_data = DataFrame('"DEMO_ComplaintAnalysis"."Customer_360_Details"')
customer_data

In [ ]:
complaints_data = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Customer_360_Complaints'))
complaints_data

<hr style='height:1px;border:none'>
<p style = 'font-size:18px;font-family:Arial'><b>4.2 Sentiment Analysis</b></p>

<p style = 'font-size:16px;font-family:Arial'>Extract the sentiment (positive, negative, neutral) using in-database functions that can execute in-platform or call out to Large Language Models of choice.</p>

<p style = 'font-size:16px;font-family:Arial'>A simple method call will extract the sentiment for patient comments <b>in-database</b> using the desired LLM and CSP provider.</p>

In [ ]:
tdf_sentiment = obj.analyze_sentiment(column = 'consumer_complaint_narrative', 
                                      data = complaints_data)[['Customer_ID','Sentiment','consumer_complaint_narrative']]
tdf_sentiment

<hr style='height:1px;border:none'>
<p style = 'font-size:18px;font-family:Arial'><b>4.3 Topic Modeling</b></p>


<p style = 'font-size:16px;font-family:Arial'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts. In this case, we are looking for specific topics to drive downstream analytics.</p>
<p style = 'font-size:16px;font-family:Arial'>Provide a list of topics to use for classification.</p>

In [ ]:
tdf_topics = obj.classify(column = 'consumer_complaint_narrative', 
                          data = complaints_data,
                          labels = ['Mortgage Application',
                                    'Payment Trouble',
                                    'Mortgage Closing',
                                    'Report Inaccuracy',
                                    'Payment Struggle'])[['Customer_ID','Labels','consumer_complaint_narrative']]
tdf_topics

<hr style='height:1px;border:none'>
<p style = 'font-size:18px;font-family:Arial'><b>4.3 Summarization</b></p>


<p style = 'font-size:16px;font-family:Arial'>The summarize method uses the model to summarize the text in the specified column of a database table. It generates an abstractive summary for the input using different levels. The conciseness of the summary can be adjusted using different levels. Higher levels yield more concise summaries.</p>

In [ ]:
tdf_summary = obj.summarize(column = 'consumer_complaint_narrative', 
                          data = complaints_data,
                          levels = 2 # higher values provide more concise summary
                           )[['Customer_ID','Summary','consumer_complaint_narrative']]

tdf_summary

<hr style='height:2px;border:none'>
<b style = 'font-size:20px;font-family:Arial'>5. Consolidated data in-database Customer360</b>

<p style = 'font-size:16px;font-family:Arial'>The developer can now perform simple joins on the data <b>in-database</b> to provide a consolidated view of the complaint summary, sentiment, topic label, and customer information.</p>

In [ ]:
tdf_combined = customer_data.join(tdf_topics.drop('consumer_complaint_narrative', axis = 1), on = ['"Customer Identifier" = Customer_ID']).drop('Customer_ID', axis = 1)
tdf_combined = tdf_combined.join(tdf_summary.drop('consumer_complaint_narrative', axis = 1), on = ['"Customer Identifier" = Customer_ID']).drop('Customer_ID', axis = 1)
tdf_combined = tdf_combined.join(tdf_sentiment.drop('consumer_complaint_narrative', axis = 1), on = ['"Customer Identifier"= Customer_ID']).drop('Customer_ID', axis = 1)

In [ ]:
tdf_combined[['Customer Identifier','Name','Sentiment','Labels','Summary']]

<hr style='height:1px;border:none'>
<p style = 'font-size:18px;font-family:Arial'><b>5.1 Persist the dataset</b></p>
<p style = 'font-size:16px;font-family:Arial'>Simple python methods will materialize the data to a permanent table if desired.</p>

In [ ]:
copy_to_sql(tdf_combined, table_name = 'Customer360', temporary = True, if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025, 2026. All Rights Reserved
        </div>
    </div>
</footer>